In [1]:
from mpl_toolkits.basemap import Basemap  # import Basemap matplotlib toolkit
import numpy as np
import matplotlib.pyplot as plt
import pygrib # import pygrib interface to grib_api
import pandas as pd
import os
import sys
import math
import folium
import csv 

%matplotlib inline

root = os.getcwd()
root_data = root + "/grib_data"
root_URL = "http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/"
year = "2018"
month = "01"
day = "01"

def getgrib(year,month,day):
    file_name = "Z__C_RJTD_"+str(year+month+day)+"000000_MSM_GPV_Rjp_Lsurf_FH00-15_grib2.bin"
    file_url = root_URL\
                +str(year + "/" + month + "/" + day +"/" )\
                +file_name
    root_file= "grib_data/"+file_name
    if os.path.isfile(root_file) == False:
            command = "wget -P grib_data/ "+ file_url
            os.system(command)

    getgrib_output = pygrib.open(root_file)
    return getgrib_output

def getrootgrib(year,month,day):
    file_name = "Z__C_RJTD_"+str(year+month+day)+"000000_MSM_GPV_Rjp_Lsurf_FH00-15_grib2.bin"    
    return "grib_data/"+file_name

In [2]:
print(os.path.isfile( getrootgrib(year,month,day) ))

True


In [3]:
grbs = getgrib(year,month,day)

In [4]:
print(os.path.isfile( getrootgrib(year,month,day) ))

True


In [5]:
#check parameters in gribfile
for grb in grbs:
    print(grb)

1:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 0 hrs:from 201801010000
2:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201801010000
3:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 201801010000
4:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 201801010000
5:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 0 hrs:from 201801010000
6:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 0 hrs:from 201801010000
7:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201801010000
8:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201801010000
9:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201801010000
10:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 2

133:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 11 hrs:from 201801010000
134:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 11 hrs:from 201801010000
135:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 11 hrs:from 201801010000
136:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 11 hrs:from 201801010000
137:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 11 hrs:from 201801010000
138:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 11 hrs:from 201801010000
139:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 11 hrs:from 201801010000
140:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 11 hrs:from 201801010000
141:Total precipitation:kg m-2 (accum):regular_ll:surface:level 0:fcst time 10-11 hrs (accum):from 201801010000
142:Downward short-wave radiation flux:W m**-2

In [6]:
#ID 27 is Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 2 hrs:from 201801010000
# selectメソッドで指定した要素(forecastTime=0)にマッチする要素の配列が返ってくる。
# forecastTime=0の要素は複数あるが先頭([0])はTempratureが入っている
grb = grbs.select(forecastTime=0)[5]

# lats,lonsは二次元配列で緯度経度が入っている
lats, lons = grb.latlons()

# 中に含まれているデータ(temprature)を二次元のndarrayで取り出す
temperature = grb.values

#一次元に変換
# 二次元のままでは描画できない
# 2次元の格子点を1次元配列にならす
flat_lats= np.ravel(lats)
flat_lons= np.ravel(lons)
#華氏を摂氏に変換
flat_temperature= (np.ravel(temperature)-32)/1.8

In [7]:
print(type(lats), len(lats))
print(type(lons), len(lons))
print(type(temperature), len(temperature))

<class 'numpy.ndarray'> 505
<class 'numpy.ndarray'> 505
<class 'numpy.ndarray'> 505


In [8]:
pd_all = {'temperature':flat_temperature, 'lons':flat_lons, 'lats':flat_lats}
df_all = pd.DataFrame(pd_all)

mamy_map_temperature=df_all["temperature"].values
mamy_map_lat=df_all["lats"].values
mamy_map_lon=df_all["lons"].values

df_all[:10]

,temperature,lons,lats
0,24.523277,120.0000,47.6
1,26.398277,120.0625,47.6
2,26.658694,120.1250,47.6
3,25.842722,120.1875,47.6
4,26.120500,120.2500,47.6
5,26.450361,120.3125,47.6
6,26.380916,120.3750,47.6
7,26.901750,120.4375,47.6
8,27.214250,120.5000,47.6
9,26.867027,120.5625,47.6


In [9]:
#緯度経度の範囲を選択
east_lon=131.775
west_lon=130.0
north_lat=34
south_lat=33.0

In [10]:
loops=len(mamy_map_temperature)

In [11]:
map_temperature=[0]
map_lat=[0]
map_lon=[0]
k=0
for i in range(0,loops):
    if west_lon < mamy_map_lon[i] and mamy_map_lon[i] < east_lon:
        if south_lat < mamy_map_lat[i] and mamy_map_lat[i] < north_lat:
            map_temperature.append(mamy_map_temperature[i])
            map_lat.append(mamy_map_lat[i])
            map_lon.append(mamy_map_lon[i])
            k=k+1

In [12]:
print(k)
print(len(map_temperature))
print(type(map_temperature))

##一列目を削除する
map_temperature.pop(0)
map_lat.pop(0)
map_lon.pop(0)


print(len(map_temperature))
print(type(map_temperature))

560
561
<class 'list'>
560
<class 'list'>


In [13]:
#小数点第一で四捨五入
map_temperature=np.round(map_temperature,1)

pd_data = {'temperature':map_temperature, 'lats':map_lat, 'lons':map_lon}
df = pd.DataFrame(pd_data)

map_temperature=df["temperature"].values
map_lat=df["lats"].values
map_lon=df["lons"].values
df[:10]

,temperature,lats,lons
0,14.4,33.95,130.0625
1,14.3,33.95,130.1250
2,14.2,33.95,130.1875
3,13.9,33.95,130.2500
4,13.7,33.95,130.3125
5,13.5,33.95,130.3750
6,13.4,33.95,130.4375
7,13.2,33.95,130.5000
8,13.2,33.95,130.5625
9,13.2,33.95,130.6250


In [14]:
#lat-long of focus area and zoom start
map = folium.Map([33.518022,130.471583], zoom_start=12)

# Mapbox makes some nice map tiles,
# Stamen also produce some cool map tiles which typically work at all zoom levels.
tile = folium.TileLayer('Stamen Terrain').add_to(map)

In [15]:
#adding marker and popup of temperature
for i in range(0,len(map_temperature)):
    folium.Marker(location=[float(map_lat[i])  ,float(map_lon[i])],\
                  popup="temperature ="+str(map_temperature[i])).add_to(map)
    
#we can change tiles with help of LayerConrol
folium.LayerControl().add_to(map)
map

In [16]:
from folium import plugins
#lat-long of focus area and zoom start
map2 = folium.Map([33.518022,130.471583], zoom_start=12)

# Mapbox makes some nice map tiles,
# Stamen also produce some cool map tiles which typically work at all zoom levels.
tile = folium.TileLayer('Stamen Terrain').add_to(map2)


#adding marker and popup of temperature
for i in range(0,len(map_temperature)):
    
    icon_number = plugins.BeautifyIcon(
    border_color='#00ABDC',
    text_color='#00ABDC',
    border_width=2,
    number=str(int(map_temperature[i])),
    inner_icon_style='margin-top:0;')
    
    folium.Marker(location=[float(map_lat[i])  ,float(map_lon[i])],\
                  popup="temperature ="+str(map_temperature[i]),\
                 icon=icon_number).add_to(map2)
    
#we can change tiles with help of LayerConrol
folium.LayerControl().add_to(map2)

map2_name="Temperature Map: " + year + "/" + month + "/" + day
map2.save(outfile="map2_name")
map2